# Data Ingestion Stage

In [1]:
from collections import namedtuple
import os

In [2]:
os.chdir("../")

In [16]:
%pwd

'g:\\Alzheimer’s Disease Detection\\Alzheimer-s-Disease-Detection'

In [17]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [18]:
# First define artifacts on config.yaml

# config entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


# Add into config constructor __init__.py -> DataIngestionConfig

In [19]:
from alzheimer.constants import *
from alzheimer.utils import read_yaml, create_directories

In [20]:
# Add on configuration.py

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [21]:
import os
import urllib.request as request
import zipfile
from alzheimer import logger
from alzheimer.utils.common import get_size

In [22]:
# Add to components

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [25]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-06-21 01:38:51,414: INFO: __init__: yaml file: config\config.yaml loaded successfully]
[2023-06-21 01:38:51,417: INFO: __init__: yaml file: params.yaml loaded successfully]
[2023-06-21 01:38:51,420: INFO: __init__: created directory at: artifacts]
[2023-06-21 01:38:51,423: INFO: __init__: created directory at: artifacts/data_ingestion]
[2023-06-21 01:38:52,766: INFO: 2686703264: artifacts/data_ingestion/data.zip download! with following info: 
Server: GitHub.com
Date: Tue, 20 Jun 2023 20:08:51 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"6fa5b24e63185b30fe9538e6f7ecd8b4"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-downgrade
Content-Security-Policy: default-src 'none'; base-uri 'self'; block-

BadZipFile: File is not a zip file

In [24]:
%pwd

'g:\\Alzheimer’s Disease Detection\\Alzheimer-s-Disease-Detection'